In [49]:
#importing required libraries

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

import pandas as pd
import numpy as np

import time, sys
import tqdm.notebook as tqdm

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

import warnings
warnings.filterwarnings('ignore')

In [33]:
# importing webdriver
from selenium import webdriver

In [35]:
# Urls to collect data from
url1 = "https://www.cars24.com/"
url2 = "https://www.cardekho.com/"
url3 = "https://www.ola.cars/"
url4 = "https://www.olx.in/cars_c84"

# collecting data ----> cars24 

In [146]:
def cars24_search(search_term,location):
    """Generate a URL for search term with location"""
    
    LOC = {'Delhi': '110001', 'Noida': '201301', 'Gurgoan': '122001', 'Mumbai' : '400001', 'Pune': '411001', 
           'Banglore': '560001','Hyderabad': '500001', 'Chennai': '600001', 'Kolkata': '700001', 'Ahmedabad': '380001' }
    
    search_term = search_term.replace(' ','%20')
    template = f'https://www.cars24.com/buy-used-car?sort=P&search={search_term}&storeCityId=8597&pinId={LOC[location]}'
    
    return template

In [152]:
def cars24_scrapping():
    """ 
    Function to scrap datas from cars 24.
    
    """
      
    print("SCRAPPING THE REQUIRED DETAILS")
    driver = webdriver.Chrome(r'../Car Price Prediction/chromedriver')
    driver.maximize_window()
    
    LOC = {'Delhi': '110001', 'Noida': '201301', 'Gurgoan': '122001', 'Mumbai' : '400001', 'Pune': '411001', 
           'Banglore': '560001','Hyderabad': '500001', 'Chennai': '600001', 'Kolkata': '700001', 'Ahmedabad': '380001' }
    
    location = [keys for keys in LOC.keys()]
    
    CARS= ['SUV', 'Sedan', 'Hatchback', 'Luxury Sedan', 'Luxury SUV']
    
    URL = [cars24_search(j,i) for i in location for j in CARS]
    
    LOCATION = []
    MNF_YEAR = []
    BRAND = []
    MODEL = []
    VARIANT = []
    driven_KM = []
    NOOF_owners = []
    fuel_type = []
    PRIZE = []


    for i in URL:
        driver.get(i)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        time.sleep(10)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'class' : '_1l4fi'})

        description = []
        description1 = []
        description2 = []
        price = []
    
        for i in results:
            description.append(i.find('h2', {'class': "_3FpCg"}).text)
            description1.append(i.find('p', {'class': "cvakB"}).text)
            description2.append(i.find('ul', {'class': "bVR0c"},'li').text)
            price.append(i.find('div', {'class': "_7udZZ"}).text)
            LOCATION.append(driver.find_element_by_xpath("//p[@class='_1OR7d']/label").text)
    
    
        year = [i.split(' ',2)[0] for i in description]
        brand = [i.split(' ',2)[1] for i in description]
        D1= [i.split(' ',2)[-1] for i in description]

        for i in price:
            PRIZE.append(i)

        for i in year:
            MNF_YEAR.append(i)
        for i in brand:
            BRAND.append(i)
    
        D2 = [i.rsplit(' ',1)[0] for i in description1]
        variant = [i.split(' ') [-1]for i in description1]
        for i in variant:
            VARIANT.append(i)
    
        D3 = [D1[i]+' '+D2[i] for i in range(0,len(D1))]
    
        for i in D3:
            MODEL.append(i)
    
        dummy1 = [i.split('km')[-1]for i in description2]
        kms = [i.split('km')[0] for i in description2]
        kms = [i+'km' for i in kms]

        owners = [i.split('Owner')[0] for i in dummy1]
        owners = [i+'Owner' for i in owners]
        fuel = [i.split('Owner')[-1] for i in dummy1]

        for i in kms:
            driven_KM.append(i)

        for i in owners:
            NOOF_owners.append(i)

        for i in fuel:
            fuel_type.append(i)

    driver.close()
        
    used_cars = pd.DataFrame({"LOCATION" : LOCATION,
                              "MNF_YEAR" : MNF_YEAR,
                              "BRAND" : BRAND,
                              "MODEL" : MODEL,
                              "VARIANT" : VARIANT,
                              "DRIVEN_KM" : driven_KM,
                              "FUELTYPE" : fuel_type,
                              "NOOF_OWNERS" : NOOF_owners,
                              "PRICE" : PRIZE
                             })
    
    used_cars.to_csv("used_cars.csv",index=False) #Creating CSV
    print("SCRAPPED DETAILS ARE STORED AS 'used_cars.csv'")
    
    return used_cars

In [153]:
user_cars = cars24_scrapping()

SCRAPPING THE REQUIRED DETAILS
SCRAPPED DETAILS ARE STORED AS 'used_cars.csv'


## collecting data ----> cardekho used cars

In [176]:
#response code
page = requests.get(url2)
print("Legality Response number from Car Dekho URL is:", page) # response output code of the webpage
soup = BeautifulSoup(page.content)

Legality Response number from Car Dekho URL is: <Response [403]>


##### for 403 response code we cannot perform web scraping in this stage and therefore we shall ignore the further process.

## collecting data ---> ola used cars

In [786]:
#response code
page = requests.get(url3)
print("Legality Response number from Ola Cars URL is:", page) # response output code of the webpage
soup = BeautifulSoup(page.content)

Legality Response number from Ola Cars URL is: <Response [200]>


##### Even though the legality response is 200, we are unable to inspect the webpage since right click is disabled on the webpage, making web scraping is impossible.

## collecting data ---> olx used cars

In [939]:
#response code
page = requests.get(url4, headers=headers)
print("Legality Response number from Olx Cars URL is:", page) # response output code of the webpage
soup = BeautifulSoup(page.content, 'html.parser')

Legality Response number from Olx Cars URL is: <Response [200]>


In [940]:
# As we can see that the resoponse code we got 200 so we can grab the data.

driver = webdriver.Chrome(r'../Car Price Prediction/chromedriver') # web drive
driver.get(url4) # url4 is olx link
driver.maximize_window()

# variables --> creating empty lists
location = []
brand =[]
year = []
model = []
kms_driven = []
model = []
variant = []
km = []
fuel = []
owners = []
price = []

# getting the URL of the cars
for i in range(0,500):
    url=driver.find_elements_by_xpath("//li[@class='_31j8e']/a")
    for i in url:
        prod_URL.append(i.get_attribute('href'))
    try:
        next_btn=driver.find_element_by_xpath("//button[@class='rui-39-wj rui-3evoE rui-1JPTg']").click()
    except NoSuchElementException:
        pass
    
    
for i in prod_URL:
    driver.get(i) 

    
    #extracting location
    loc = driver.find_elements_by_xpath("//div[@class='_1l939']/div[2]/div[2]/div[2]")

    try:
        for i in loc:
            t = i.text
            location.append(t)
    except NoSuchElementException as e:
        location.append('-')
        pass
        
    #Extracting brand and year by using only one xpath
    brand_year= driver.find_elements_by_xpath("//div[@class='_35xN1']")

    try:
        for i in brand_year:
            t = i.text.split('(')[0]
            brand.append(t)
    except NoSuchElementException as e:
        brand.append('-')
        pass
    
    # extracting year

    try:
        for i in brand_year:
            y = i.text.replace('(','')
            y = y.replace(')','').split(' ')
            y = ''.join(y[-1])
            year.append(y)
    except NoSuchElementException as e:
        year.append('-')
        pass
        
    #extracting model
    mod =driver.find_elements_by_xpath("//*[@id='container']/main/div/div/div/div[5]/div[1]/div[1]/div[1]/div/div[3]")

    try:
        for i in mod:
            model.append(i.text)
    except NoSuchElementException as e:
        model.append('-')
        pass
    
    
    #variant
    var = driver.find_elements_by_xpath("//div[@class='aOxkz']/div[3]/div")

    try:
        for i in var:
            variant.append(i.text)
    except NoSuchElementException as e:
        variant.append('-')
        pass
        
    #Extracting km driven
    km_driven = driver.find_elements_by_xpath("//div[@class='aOxkz']/div[2]/div")
    
    try:
        for i in km_driven:
            km.append(i.text)
    except NoSuchElementException as e:
        km.append('-')
        pass
    
    #extracting fuel 
    oil =driver.find_elements_by_xpath("//div[@class='aOxkz']/div[1]/div")

    try:
        for i in oil:
            fuel.append(i.text)
    except NoSuchElementException as e:
        fuel.append('-')
        pass
        
    #no of owners
    noofowners = driver.find_elements_by_xpath("//div[@class='_1l939']/div[1]/div[2]/div[2]")

    try:
        for i in noofowners:
            owners.append(i.text)
    except NoSuchElementExcpetion as e:
        owners.append('-')
        pass


    #price
    cost = driver.find_elements_by_xpath("//div[@class='_3FkyT']")

    try:
        for i in cost:
            price.append(i.text.strip('₹ '))
    except NoSuchElementExcpetion as e:
        price.append('-')
        pass

In [942]:
#checking collected data size in each column
len(location),len(year),len(brand),len(model),len(variant),len(km), len(fuel), len(owners),len(price)

(4526, 4526, 4526, 4526, 4526, 4526, 4526, 4526, 4526)

In [958]:
# Creating dataframe
used_cars = pd.DataFrame({"LOCATION" : location,
                          "MNF_YEAR" : year,
                          "BRAND" : brand,
                          "MODEL" : model,
                          "VARIANT" : variant,
                          "DRIVEN_KM" : km,
                          "FUELTYPE" : fuel,
                          "NOOF_OWNERS" : owners,
                          "PRICE" : price})

In [961]:
#Saving to csv file
used_cars.to_csv("Used_Car_olx_Data.csv")